In [14]:
class NetworkNode:
    ID = None
    latitude = None
    longitude = None

class FogCloudNode(NetworkNode):
    power_level = None
    model = None
    processing_capacity = None
    memory_capacity = None
    cost = None

class FogNode(FogCloudNode):
    pass

class CloudNode(FogCloudNode):
    pass

class Service:
    service_type = None
    processing_demand = None
    memory_demand = None
    number_of_bits = None
    lifetime = None

class Sensor(NetworkNode):
    services = None
    reachable_fog_nodes = None


None
2
